# * *PRÁCTICA FINAL SOTO ANÁLISIS DE SENTIMIENTOS* *

In [25]:
import os
import re
import sys
import warnings
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from google.protobuf import text_format

import spacy
import numpy as np
import pandas as pd
import emoji
import spacymoji
import codecs
from spacymoji import Emoji

sys.path.append(os.path.realpath('../'))
# from scripts.preprocess_data import Preprocess

warnings.filterwarnings('ignore')

PATH_DATA = "../"
DEV_FILE = "dev/dev.tsv"
TRAIN_FILE = "Corpus/train.tsv"

In [26]:
def give_emoji_free_text(text):
    #return emoji.get_emoji_regexp().sub(u'', text) # requiere que no sea string
    text2 = text.encode('ascii', 'ignore').decode()
    return text2

In [27]:
def basic_processing(X):
    

    tweets_clean = []
    
    nlp = spacy.load("es_core_news_sm")

    for pos, sen in enumerate(X):
        #codifica y pasa de tipo byte a tipo str unicode
        tweet = sen.decode()
        tweet = str(tweet)
        tweet = give_emoji_free_text(tweet)
        
        #tweet = re.sub('user', '', tweet)  # remove tweeted at (menciones)
        #tweet = re.sub('(#\s+[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hashtags at (hashtag)
        tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hashtags + espacio at (hashtag con espacio)
        doc = nlp(tweet)

            #obtenemos la frase lematizada 
        lista_tweet=[]
        newSentence=''
        for token in doc:
            if (
                not token.is_punct
                #not token.is_emoji
                #and not token.like_num
                and not token.is_stop
                and not token.like_url
                and not token.is_space
                and not token.pos_ == "SYM"
                #and not token.pos_ == "DET"
                and not token.pos_ == "X"
                #and not token.pos_ == "SCONJ"
                #and not token.pos_ == "CONJ"
                #and not token.pos_ == "ADP"
                #and not token.pos_ == "INTJ"
                and not token.pos_ == "NUM"
                #and not token.pos_ == "PRON"
                #and not token.pos_ == "PROPN"
            ):
                newSentence = ' '.join([newSentence,token.lemma_.lower()])
                #lista_tweet.append(newSentence)
        tweets_clean.append(newSentence)
    print(tweets_clean)
    return tweets_clean

In [28]:
def processTweetsTSV(corpus_path, trainingFile):
    if (not os.path.exists(corpus_path)):
        os.mkdir(corpus_path)

    df_dev = pd.read_csv(trainingFile, sep="\t", usecols = ['id', 'tweet','emotion'])

    anger_path = corpus_path+'//ANGER'
    disgust_path = corpus_path+"//DISGUST"
    fear_path = corpus_path+"//FEAR"
    joy_path = corpus_path+"//JOY"
    sadness_path = corpus_path + "//SADNESS"
    surprise_path = corpus_path + "//SURPRISE"
    others_path = corpus_path + "//OTHERS"

    if (not os.path.exists(anger_path)):
            os.makedirs(anger_path)
            os.makedirs(disgust_path)
            os.makedirs(fear_path)
            os.makedirs(joy_path)
            os.makedirs(sadness_path)
            os.makedirs(surprise_path)
            os.makedirs(others_path)

    #print(df_dev)
    df_dev.to_csv("filterDataTrain.csv")

    with open('filterDataTrain.csv', mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, quotechar='"', delimiter=',',
                                    quoting=csv.QUOTE_ALL, skipinitialspace=True)
        path = ''
        for row in csv_reader:
            #print(row)
            if row["emotion"] == "others":
                path= others_path
            elif row["emotion"] == "anger":
                path= anger_path
            elif row["emotion"] == "disgust":
                path= disgust_path
            elif row["emotion"] == "fear":
                path= fear_path
            elif row["emotion"] == "joy":
                path = joy_path
            elif row["emotion"] == "sadness":
                path = sadness_path
            elif row["emotion"] == "surprise":
                path = surprise_path

            # Creating new file
            f=open(path+"//"+row['id']+".txt","w", encoding='utf-8')
            f.write(row['tweet'])
            f.close()

In [29]:
def classifier(corpus_path):

    # CorpusTrain and subfolders (categories) must exist
    tweets_data = load_files(corpus_path)
    X, y = tweets_data.data, tweets_data.target

    documents = basic_processing(X)

    vectorizer = CountVectorizer()
    #vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1, 1))
    X = vectorizer.fit_transform(documents).toarray()

    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(X).toarray()

    # The data is divided into 20% test set and 80% training set.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    #clf = MultinomialNB().fit(X_train, y_train)
    #clf = BernoulliNB().fit(X_train, y_train)
    #clf = GaussianNB().fit(X_train, y_train)
    #clf = ComplementNB().fit(X_train, y_train)
    #clf = CategoricalNB().fit(X_train, y_train)
    #clf = DecisionTreeClassifier().fit(X_train, y_train)
    #clf = ExtraTreeClassifier().fit(X_train, y_train)
    clf = LinearSVC().fit(X_train, y_train)
    #clf = SVC().fit(X_train, y_train)
    #clf = KNeighborsClassifier().fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(confusion_matrix(y_test,y_pred))
    print("------------------------------------------")
    print(classification_report(y_test,y_pred))
    print("------------------------------------------")
    print("accuracy",accuracy_score(y_test, y_pred))

In [30]:
if __name__ == "__main__":
    trainingFile = "Corpus/train.tsv"
    corpus_path = "CorpusTraining"

    processTweetsTSV(corpus_path, trainingFile)
    # entrenar el clasificador y evaluar el rendimiento
    classifier(corpus_path)

[' user chavista pensar as va a salir pobre ajajaj', ' mvp episodio aprovechar noche previo a batalla user', ' user enredado fcil rival user', ' secretario oea user manifest apoyo a adhesin militar a constitucin necesario pleno respaldo proceso transicin democrtico forma pacfico user user', ' brutal cifra tanto prdida y moral gente slo francés mundo entero perdido cono grande noche oracin ayudar a alma francés semejante acontecimiento est luto', ' y dormir joder', ' y perder madrid felicidad', ' creer ms messi so enorme pulgo', ' hospital universitario ceuta celebrar ms pequeo', ' rbitro campar real madrid puto vendido penal a favor barcelona', ' amar ao y pensamiento a tocar dcada configurar y ms consciente millón adulto junto', ' presentar a santa arya stark', ' vea transmisin actualidad radio am', ' captulo hora y pedo', ' rea tomar', ' 70 despus gol levante acercar ms arco barcelona equipo cul tranquilizar él y volver a pelota empezar a jugar', ' user mencionado narcotraficante fra

# PRUEBAS

In [19]:
s1 = "Hi \xF0\x9F\x98\x81 How is your 🙈 and 😌. Have a nice weekend 💕👭👙"

In [18]:
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(u'', text)

print (give_emoji_free_text(s1))

Hi ð How is your  and . Have a nice weekend 


In [13]:
import re
#s2 = " b'os presentar a santa arya stark . \\xf0\\x9f\\x99\\x8f"
s2 = b'USER Que mal eres chavista? ! Por pensar as\xc3\xad no vas a salir de pobre ajajaj. #VenezuelaLibre #MaduroNoEsNadie'

In [4]:
def give_emoji_free_text(text):
 #   return emoji.get_emoji_regexp().sub(u'', text)
    emoji_pattern = re.compile(pattern = "["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

print (give_emoji_free_text(s2))

 b'os presentar a santa arya stark . ð


In [14]:
text = s2.encode('ascii', 'ignore').decode()

print(text)

AttributeError: 'bytes' object has no attribute 'encode'

In [3]:
text2 = text.encode('ascii', 'ignore').decode()

print(text2)

 b'os presentar a santa arya stark . \xf0\x9f\x99\x8f
